In [1]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import roc_auc_score

# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import normalize
from sklearn.model_selection import cross_val_score

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
df_train = pd.read_csv('application_train.csv')
df_test = pd.read_csv('application_test.csv')

### Обзор данных

In [4]:
df_train.head()

SK_ID_CURR  TARGET NAME_CONTRACT_TYPE CODE_GENDER FLAG_OWN_CAR  \
0      100002       1         Cash loans           M            N   
1      100003       0         Cash loans           F            N   
2      100004       0    Revolving loans           M            Y   
3      100006       0         Cash loans           F            N   
4      100007       0         Cash loans           M            N   

  FLAG_OWN_REALTY  CNT_CHILDREN  AMT_INCOME_TOTAL  AMT_CREDIT  AMT_ANNUITY  \
0               Y             0          202500.0    406597.5      24700.5   
1               N             0          270000.0   1293502.5      35698.5   
2               Y             0           67500.0    135000.0       6750.0   
3               Y             0          135000.0    312682.5      29686.5   
4               Y             0          121500.0    513000.0      21865.5   

   ...  FLAG_DOCUMENT_18 FLAG_DOCUMENT_19 FLAG_DOCUMENT_20 FLAG_DOCUMENT_21  \
0  ...                 0                0                0                0   
1  ...                 0                0                0                0   
2  ...                 0                0                0                0   
3  ...                 0                0                0                0   
4  ...                 0                0                0                0   

  AMT_REQ_CREDIT_BUREAU_HOUR AMT_REQ_CREDIT_BUREAU_DAY  \
0                        0.0                       0.0   
1                        0.0                       0.0   
2                        0.0                       0.0   
3                        NaN                       NaN   
4                        0.0                       0.0   

   AMT_REQ_CREDIT_BUREAU_WEEK  AMT_REQ_CREDIT_BUREAU_MON  \
0                         0.0                        0.0   
1                         0.0                        0.0   
2                         0.0                        0.0   
3                         NaN                        NaN   
4                         0.0                        0.0   

   AMT_REQ_CREDIT_BUREAU_QRT  AMT_REQ_CREDIT_BUREAU_YEAR  
0                        0.0                         1.0  
1                        0.0                         0.0  
2                        0.0                         0.0  
3                        NaN                         NaN  
4                        0.0                         0.0  

[5 rows x 122 columns]

In [5]:
df_test.head()

SK_ID_CURR NAME_CONTRACT_TYPE CODE_GENDER FLAG_OWN_CAR FLAG_OWN_REALTY  \
0      100001         Cash loans           F            N               Y   
1      100005         Cash loans           M            N               Y   
2      100013         Cash loans           M            Y               Y   
3      100028         Cash loans           F            N               Y   
4      100038         Cash loans           M            Y               N   

   CNT_CHILDREN  AMT_INCOME_TOTAL  AMT_CREDIT  AMT_ANNUITY  AMT_GOODS_PRICE  \
0             0          135000.0    568800.0      20560.5         450000.0   
1             0           99000.0    222768.0      17370.0         180000.0   
2             0          202500.0    663264.0      69777.0         630000.0   
3             2          315000.0   1575000.0      49018.5        1575000.0   
4             1          180000.0    625500.0      32067.0         625500.0   

   ... FLAG_DOCUMENT_18 FLAG_DOCUMENT_19 FLAG_DOCUMENT_20 FLAG_DOCUMENT_21  \
0  ...                0                0                0                0   
1  ...                0                0                0                0   
2  ...                0                0                0                0   
3  ...                0                0                0                0   
4  ...                0                0                0                0   

  AMT_REQ_CREDIT_BUREAU_HOUR  AMT_REQ_CREDIT_BUREAU_DAY  \
0                        0.0                        0.0   
1                        0.0                        0.0   
2                        0.0                        0.0   
3                        0.0                        0.0   
4                        NaN                        NaN   

   AMT_REQ_CREDIT_BUREAU_WEEK  AMT_REQ_CREDIT_BUREAU_MON  \
0                         0.0                        0.0   
1                         0.0                        0.0   
2                         0.0                        0.0   
3                         0.0                        0.0   
4                         NaN                        NaN   

   AMT_REQ_CREDIT_BUREAU_QRT  AMT_REQ_CREDIT_BUREAU_YEAR  
0                        0.0                         0.0  
1                        0.0                         3.0  
2                        1.0                         4.0  
3                        0.0                         3.0  
4                        NaN                         NaN  

[5 rows x 121 columns]

### Пропущенные данные

In [6]:
mis_val_percent = (100 * pd.isnull(df_train).sum() / len(df_train)).sort_values(ascending = False)[:15]

In [7]:
for col in mis_val_percent.index:
    if(mis_val_percent[col] > 80):
        del df_train[col]
        del df_test[col]

In [8]:
answer = pd.DataFrame()
answer['SK_ID_CURR'] = df_test['SK_ID_CURR']

y = df_train['TARGET']
del df_train['TARGET']

del df_test['SK_ID_CURR']
del df_train['SK_ID_CURR']

In [9]:
num_cols = df_train.select_dtypes(exclude = ["object"]).columns
categ_cols = df_train.select_dtypes(include = ["object"]).columns

In [10]:
for col in num_cols:
    df_train[col] = df_train[col].fillna(df_train[col].mean())
    df_test[col] = df_test[col].fillna(df_test[col].mean())

In [11]:
for col in categ_cols:
    df_train[col] =df_train[col].fillna('None')
    df_test[col] = df_test[col].fillna('None')

### корреляция

In [12]:
corr_matrix = df_train[num_cols].corr().abs()

upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.7)]

In [13]:
df_train = df_train.drop(to_drop, axis=1)
df_test = df_test.drop(to_drop, axis=1)

In [14]:
df = pd.concat([df_train, df_test])

In [15]:
df = pd.get_dummies(df)

df_test = df.iloc[df_train.shape[0]:,:]
df_train = df.iloc[:df_train.shape[0],:]

In [16]:
from sklearn.preprocessing import MinMaxScaler

In [17]:
def encoder(df):
    scaler = MinMaxScaler()
    numerical = df.select_dtypes(exclude = ["object"]).columns
    features_transform = pd.DataFrame(data= df)
    features_transform[numerical] = scaler.fit_transform(df[numerical])
    display(features_transform.head(n = 5))
    return df

df_train_norm = encoder(df_train)
df_test_norm = encoder(df_test)

CNT_CHILDREN  AMT_INCOME_TOTAL  AMT_CREDIT  REGION_POPULATION_RELATIVE  \
0           0.0          0.001512    0.090287                    0.256321   
1           0.0          0.002089    0.311736                    0.045016   
2           0.0          0.000358    0.022472                    0.134897   
3           0.0          0.000935    0.066837                    0.107023   
4           0.0          0.000819    0.116854                    0.392880   

   DAYS_BIRTH  DAYS_EMPLOYED  DAYS_REGISTRATION  DAYS_ID_PUBLISH  OWN_CAR_AGE  \
0    0.888839       0.045086           0.852140         0.705433     0.132539   
1    0.477114       0.043648           0.951929         0.959566     0.132539   
2    0.348534       0.046161           0.827335         0.648326     0.285714   
3    0.350846       0.038817           0.601451         0.661387     0.132539   
4    0.298591       0.038820           0.825268         0.519522     0.132539   

   FLAG_MOBIL  ...  WALLSMATERIAL_MODE_Mixed  WALLSMATERIAL_MODE_Monolithic  \
0         1.0  ...                       0.0                            0.0   
1         1.0  ...                       0.0                            0.0   
2         1.0  ...                       0.0                            0.0   
3         1.0  ...                       0.0                            0.0   
4         1.0  ...                       0.0                            0.0   

   WALLSMATERIAL_MODE_None  WALLSMATERIAL_MODE_Others  \
0                      0.0                        0.0   
1                      0.0                        0.0   
2                      1.0                        0.0   
3                      1.0                        0.0   
4                      1.0                        0.0   

   WALLSMATERIAL_MODE_Panel  WALLSMATERIAL_MODE_Stone, brick  \
0                       0.0                              1.0   
1                       0.0                              0.0   
2                       0.0                              0.0   
3                       0.0                              0.0   
4                       0.0                              0.0   

   WALLSMATERIAL_MODE_Wooden  EMERGENCYSTATE_MODE_No  \
0                        0.0                     1.0   
1                        0.0                     1.0   
2                        0.0                     0.0   
3                        0.0                     0.0   
4                        0.0                     0.0   

   EMERGENCYSTATE_MODE_None  EMERGENCYSTATE_MODE_Yes  
0                       0.0                      0.0  
1                       0.0                      0.0  
2                       1.0                      0.0  
3                       1.0                      0.0  
4                       1.0                      0.0  

[5 rows x 209 columns]

CNT_CHILDREN  AMT_INCOME_TOTAL  AMT_CREDIT  REGION_POPULATION_RELATIVE  \
0          0.00          0.024654    0.238037                    0.257380   
1          0.00          0.016440    0.080785                    0.491855   
2          0.00          0.040054    0.280965                    0.260854   
3          0.10          0.065721    0.695297                    0.361760   
4          0.05          0.034920    0.263804                    0.135340   

   DAYS_BIRTH  DAYS_EMPLOYED  DAYS_REGISTRATION  DAYS_ID_PUBLISH  OWN_CAR_AGE  \
0    0.333427       0.039545           0.782059         0.872086     0.159271   
1    0.399339       0.033953           0.615631         0.744329     0.159271   
2    0.288794       0.033982           0.908313         0.448173     0.067568   
3    0.628269       0.040755           0.915690         0.337114     0.159271   
4    0.680685       0.039905           0.831380         0.328607     0.216216   

   FLAG_MOBIL  ...  WALLSMATERIAL_MODE_Mixed  WALLSMATERIAL_MODE_Monolithic  \
0         1.0  ...                       0.0                            0.0   
1         1.0  ...                       0.0                            0.0   
2         1.0  ...                       0.0                            0.0   
3         1.0  ...                       0.0                            0.0   
4         1.0  ...                       0.0                            0.0   

   WALLSMATERIAL_MODE_None  WALLSMATERIAL_MODE_Others  \
0                      0.0                        0.0   
1                      1.0                        0.0   
2                      1.0                        0.0   
3                      0.0                        0.0   
4                      1.0                        0.0   

   WALLSMATERIAL_MODE_Panel  WALLSMATERIAL_MODE_Stone, brick  \
0                       0.0                              1.0   
1                       0.0                              0.0   
2                       0.0                              0.0   
3                       1.0                              0.0   
4                       0.0                              0.0   

   WALLSMATERIAL_MODE_Wooden  EMERGENCYSTATE_MODE_No  \
0                        0.0                     1.0   
1                        0.0                     0.0   
2                        0.0                     0.0   
3                        0.0                     1.0   
4                        0.0                     0.0   

   EMERGENCYSTATE_MODE_None  EMERGENCYSTATE_MODE_Yes  
0                       0.0                      0.0  
1                       1.0                      0.0  
2                       1.0                      0.0  
3                       0.0                      0.0  
4                       1.0                      0.0  

[5 rows x 209 columns]

### Модель: score 0.50311

In [22]:
clf = LogisticRegression(random_state=42)

In [23]:
scores = cross_val_score(clf, df_train_norm, y, cv=5, scoring='roc_auc')

In [24]:
scores.mean()

0.7418944854131648

In [25]:
result = clf.fit(df_train_norm, y).predict(df_test_norm)

In [26]:
answer['TARGET'] = result

In [27]:
answer.to_csv('0_model.csv', header = True, index = False)